# Iterators and Generators

So far, you have seen things like

In [1]:
a = [1,2,3]
for x in a:
    print(x)

1
2
3


This looks very different from a C-style for loop where we loop over the variable index:
```C++
for (size_t i = 0; i < 3; i++) {
    printf("%d\n", i);
}
```

Or for instance, we can use something called a `range`

In [1]:
for i in range(3):
    print(i)

0
1
2


or other data types

In [2]:
d = {"hello": 1, "goodbye": 2}
for k,v in d.items():
    print(k, ' : ', v)

hello  :  1
goodbye  :  2


The key to using this sort of syntax is the concept of [iterator](https://wiki.python.org/moin/Iterator).  This is common in object-oriented programming (not just in Python), but you probably haven't seen iterators before if you've only used imperative languages.

An object is **iterable** if it implements the `__iter__` method, which is expected to return an iterator object.
An object is an **iterator** if it implements the `__next__` method, which either
1. returns the next element of the iterable object
2. raises the `StopIteration` exception if there are no more elements to iterate over

## A Basic Iterator

What if we want to replicate `range`?  

In [3]:
r = range(3)
type(r)

range

we can produce an iterator using the `iter` function

In [4]:
ri = iter(r)
type(ri)

range_iterator

we can explicitly run through the iterator using the `next` function

In [5]:
next(ri)

0

In [6]:
class my_range_iterator(object):
    def __init__(self, start, stop, stride):
        self.state = start
        self.stop = stop
        self.stride = stride
        
    def __next__(self):
        if self.state >= self.stop:
            raise StopIteration  # signals "the end"
        ret = self.state # we'll return current state
        self.state += self.stride # increment state
        return ret
        
        
# an iterable
class my_range(object):
    def __init__(self, start, stop, stride=1):
        self.start = start
        self.stop = stop
        self.stride = stride
    
    def __iter__(self):
        return my_range_iterator(self.start, self.stop, self.stride)

In [7]:
r = my_range(0,3)
type(r)

__main__.my_range

In [8]:
ri = iter(r)
type(ri)

__main__.my_range_iterator

In [9]:
next(ri)

0

In [10]:
for i in my_range(0,3):
    print(i)

0
1
2


You can also create classes that are both iterators and iterables

In [11]:
# an iterable and iterator
class my_range2(object):
    def __init__(self, start, stop, stride=1):
        self.start = start
        self.stop = stop
        self.stride = stride
        self.state = start
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.state >= self.stop:
            raise StopIteration  # signals "the end"
        ret = self.state # we'll return current state
        self.state += self.stride # increment state
        return ret

In [12]:
for i in my_range2(0,3):
    print(i)

0
1
2


## Using Iterators for Computation

Let's now use iterators for something more interesting - computing the Fibonacci numbers.

In [13]:
class FibonacciIterator(object):
    def __init__(self):
        self.a = 0 # current number
        self.b = 1 # next number
        
    def __iter__(self):
        return self
    
    def __next__(self):
        ret = self.a
        self.a, self.b = self.b, self.a + self.b # advance the iteration
        return ret

In [14]:
for i in FibonacciIterator():
    if i > 1000:
        break
    print(i)

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987


Note that we never raise a `StopIteration` exception - the iterator will just keep going if we let it.

### Exercises

## Generators

Often, a more elegant way to define an iterator is using a [generator](https://wiki.python.org/moin/Generators)

This is a special kind of iterator defined using a function instead of using classes that implement the `__iter__` and `__next__` methods.

See [this post](https://nvie.com/posts/iterators-vs-generators/) for more discussion.

In [15]:
def my_range3(state, stop, stride=1):
    while state < stop:
        yield state
        state += stride
        

Note that we use the `def` keyword instead of the `class` keyword for our declaration.  The `yield` keyword returns subsequent values of the iteration.

In [16]:
r = my_range3(0,3)
type(r)

generator

In [17]:
ri = iter(r)
type(ri)

generator

In [18]:
next(ri)

0

In [19]:
for i in my_range3(0,3):
    print(i)

0
1
2


Our Fibonacci example re-written using a generator:

In [20]:
def FibonacciGenerator():
    a = 0
    b = 1
    while True:
        yield a
        a, b = b, a + b

In [21]:
for i in FibonacciGenerator():
    if i > 1000:
        break
    print(i)


0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987


### Exercises

## zip and reverse

In [22]:
for i, a in zip([0,1,2], ['a', 'b', 'c']):
    print(i,a)

0 a
1 b
2 c


In [23]:
for i in reversed(range(3)):
    print(i)

2
1
0


## The Itertools Package

A useful package for dealing with iterators is the [itertools package](https://docs.python.org/3.8/library/itertools.html)

## Iterators for Scientific Computing

One way to use iterators for scientific computing

Iterator for power method

In [24]:
import numpy as np

def PowerMethodIterator(A, x):
    
    def RayleighQuotient(A, x):
        return np.dot(x, A @ x) / np.dot(x,x)
    
    x = x / np.linalg.norm(x)
    rq_prev = np.inf
    rq = RayleighQuotient(A, x)
    
    while True:
        # yield state: RayleighQuotient, x, and difference from previous iteration
        yield rq, x, np.abs(rq - rq_prev)
        
        # compute next iteration
        x = A @ x
        x = x / np.linalg.norm(x)
        rq_prev = rq
        rq = RayleighQuotient(A, x)
    

In [25]:
n = 100
A = np.random.randn(n, n)
A = A @ A.T
x0 = np.random.randn(n)

solver = PowerMethodIterator(A, x0)
tol = 1e-4

while True:
    rq, x, eps = next(solver)
    print(rq, eps)
    if eps < tol:
        break

84.78647511789411 inf
235.14230845970027 150.35583334180615
299.0901581710425 63.94784971134223
327.42632196144444 28.33616379040194
339.5066815684834 12.080359607038929
345.45802773278973 5.951346164306358
348.9275032625951 3.469475529805379
351.2607348995293 2.333231636934215
353.0377966940488 1.7770617945194545
354.5453002142895 1.507503520240732
355.93568840597504 1.3903881916855312
357.29133268470406 1.3556442787290166
358.6546293329733 1.3632966482692268
360.0436641675765 1.3890348346031942
361.4610192312393 1.4173550636627965
362.8991655027012 1.4381462714619033
364.3441405924098 1.4449750897086346
365.77834450515587 1.4342039127460566
367.1828184923569 1.4044739872010155
368.5391202293479 1.3563017369909858
369.8307914876373 1.2916712582894547
371.0443849619949 1.2135934743575945
372.1700390305733 1.125654068578399
373.2016326343672 1.0315936037938513
374.13659387375276 0.9349612393855864
374.9754615052754 0.8388676315226462
375.72130410324775 0.7458425979723415
376.37909031011

If we decide that we're not satisfied with convergence yet, we can resume where we left off

In [26]:
tol = 1e-6

while True:
    rq, x, eps = next(solver)
    print(rq, eps)
    if eps < tol:
        break

380.36230471717 7.720269559285953e-05
380.362369456631 6.473946098140004e-05
380.3624237448757 5.4288244712097367e-05
380.3624692691103 4.55242346220075e-05
380.36250744416355 3.817505324832382e-05
380.36253945645876 3.201229520755078e-05
380.3625663008861 2.6844427338801324e-05
380.36258881172347 2.251083736837245e-05
380.3626076885651 1.8876841636483732e-05
380.36262351806477 1.58294996595032e-05
380.36263679216904 1.3274104276206344e-05
380.3626479234059 1.1131236874462047e-05
380.3626572577068 9.33430089844478e-06
380.3626650851574 7.827450588138163e-06
380.36267164901295 6.563855549757136e-06
380.36267715325937 5.504246416876413e-06
380.36268176895163 4.6156922621776175e-06
380.36268563953087 3.870579234899196e-06
380.36268888528207 3.2457512020300783e-06
380.36269160707207 2.7217899969400605e-06
380.36269388948443 2.2824123675491137e-06
380.3626958034482 1.9139637856824265e-06
380.3626974084423 1.604994054105191e-06
380.3626987543438 1.3459015235639527e-06
380.3626998829779 1.128

Implement a class which contains the state of an optimization problem.  Make this class an iterator.

Use this for introspection of the solver.

### Exercises

Implement the game of life using an Iterator or Generator